In [24]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
from tqdm import tqdm
from small2DNet import small2DNet
from small3DNet import small3DNet
from util import add_color, colorize, colorize_gaussian, calculate_correct_loss
from colorMNist import colorMNist
import random
import colorsys
import pickle
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [2]:
def cs(x, y):
    z = torch.sum(torch.mul(x, y)).item() / (np.sqrt(torch.sum(torch.square(x)).item()) * np.sqrt(torch.sum(torch.square(y)).item()))
    return z
def ed(x, y):
    z = np.sqrt(torch.sum(torch.square(torch.torch.sub(x, y))).item())
    return z

In [12]:
# Uniform
# Layers of the model
model_layers = [8, 8, "M", 16,"M"]
# Create model
model = small2DNet(model_layers, 16)
# Load model
model.load_state_dict(torch.load('model_saves/2D/Uniform/Uniform2D.pth'))
# Put model on gpu
model.cuda()

children = [x for x in model.children()]

# print(children[0][0].weight)
uniform = []
uniformcs = []
for x in children[0][0].weight:
    uniform.append((ed(x[0], x[1]) + ed(x[0], x[2]) + ed(x[1], x[2])) / 3)
    uniformcs.append((cs(x[0], x[1]) + cs(x[0], x[2]) + cs(x[1], x[2])) / 3)
    # print(ed(x[0], x[1]), ed(x[0], x[2]), ed(x[1], x[2]))
    print(cs(x[0], x[1]), cs(x[0], x[2]), cs(x[1], x[2]))
    # break

0.4042112754503246 -0.01405491525256262 0.2655584153692246
0.7963316201942805 0.7061117372671798 0.8016470731212859
0.7189544314774989 0.09466627669464943 0.032478127683876694
-0.07088823317925103 0.13510858056582936 0.3351765047110641
0.34624104763774755 -0.18710270900223067 -0.2768761534167951
-0.2156390594508796 -0.1677460477615484 -0.12260641513290302
-0.21788042927847193 0.9388103122971455 -0.4376612000012891
-0.11693447147434596 0.3304379558502203 0.49169739248188554


In [6]:
# Uniform
# Layers of the model
model_layers = [8, 8, "M", 16,"M"]
# Create model
model = small2DNet(model_layers, 16)
# Load model
model.load_state_dict(torch.load('model_saves/2D/Uniform/Uniform2D.pth'))
# Put model on gpu
model.cuda()

children = [x for x in model.children()]

uniformflatten = []
for x in children[0][0].weight:
    # print(cosine_similarity(x[0].cpu().detach().numpy().flatten(), x[1].cpu().detach().numpy().flatten()))
    x0 = x[0].cpu().detach().numpy().flatten().reshape(1, -1)
    x1 = x[1].cpu().detach().numpy().flatten().reshape(1, -1)
    x2 = x[2].cpu().detach().numpy().flatten().reshape(1, -1)
    uniformflatten.append((cosine_similarity(x0, x1)[0][0] + cosine_similarity(x0, x2)[0][0] + cosine_similarity(x1, x2)[0][0]) / 3)
    # print(cosine_similarity(x0, x1)[0][0])

In [22]:
# Colorless
# Layers of the model
model_layers = [8, 8, "M", 16,"M"]
# Create model
model = small2DNet(model_layers, 16)
# Load model
model.load_state_dict(torch.load('model_saves/2D/Colorless/Colorless2D.pth'))
# Put model on gpu
model.cuda()
children = [x for x in model.children()]

colorless = []
colorlesscs = []
for x in children[0][0].weight:
    colorless.append((ed(x[0], x[1]) + ed(x[0], x[2]) + ed(x[1], x[2])) / 3)
    colorlesscs.append((cs(x[0], x[1]) + cs(x[0], x[2]) + cs(x[1], x[2])) / 3)
    print(np.round(cs(x[0], x[1]), 4), "\t", np.round(cs(x[0], x[2]), 4), np.round(cs(x[1], x[2]), 4))
    # break

0.4221 	 0.1435 0.0103
0.2123 	 0.6026 0.3906
0.6792 	 0.7761 0.8766
0.1557 	 0.1491 -0.2244
0.4167 	 0.768 0.3063
0.5206 	 0.1395 0.1881
0.7531 	 -0.0255 0.1464
0.5002 	 0.8842 0.3195


In [21]:
print(np.round(uniform, 4))
print(np.round(colorless, 4))
print(uniformcs)
print(colorlesscs)
print(np.sum([colorless[i] - uniform[i] for i in range(len(uniform))]))
print(np.sum([uniformcs[i] - colorlesscs[i] for i in range(len(uniformcs))]))
# print(np.sum([uniformflatten[i] - colorlessflatten[i] for i in range(len(uniformflatten))]))

[0.4589 0.4343 0.5347 0.4912 0.5612 0.5743 0.5429 0.4009]
[0.4138 0.4176 0.4152 0.4429 0.5007 0.4138 0.3645 0.373 ]
[0.2185715918556622, 0.7680301435275821, 0.2820329452853417, 0.13313228403254748, -0.03924593826042608, -0.168663840781777, 0.09442289433912815, 0.23506695895258664]
[0.1919860248267549, 0.40185692560993996, 0.7772901538835884, 0.026808929628459766, 0.4970272222020761, 0.2827624008588103, 0.291300556007501, 0.5679762976219732]
-0.6569137521952385
-1.5136614716884584


In [29]:
# Uniform
# Layers of the model
model_layers = [8, 8, "M", 16,"M"]
# Create model
model = small3DNet(model_layers, 16)
# Load model
model.load_state_dict(torch.load('model_saves/3D/Uniform/Uniform3D.pth'))
# Put model on gpu
model.cuda()

children = [x for x in model.children()]

uniformcs = []
uniformed = []
for x in children[0][0].weight:
    x0 = x[0][0].cpu().detach().numpy().flatten().reshape(1, -1)
    x1 = x[0][1].cpu().detach().numpy().flatten().reshape(1, -1)
    x2 = x[0][2].cpu().detach().numpy().flatten().reshape(1, -1)
    uniformcs.append((cosine_similarity(x0, x1)[0][0] + cosine_similarity(x0, x2)[0][0] + cosine_similarity(x1, x2)[0][0]) / 3)
    uniformed.append((euclidean_distances(x0, x1)[0][0] + euclidean_distances(x0, x2)[0][0] + euclidean_distances(x1, x2)[0][0]) / 3)
    
    print(np.round(euclidean_distances(x0, x1)[0][0], 4), np.round(euclidean_distances(x1, x2)[0][0], 4), np.round(euclidean_distances(x0, x2)[0][0], 4))

0.4066 0.324 0.3125
0.5022 0.6289 0.5233
0.4117 0.9264 0.9342
0.3989 0.4879 0.2756
0.4068 0.5828 0.7248
0.4124 0.5264 0.5036
0.4896 0.7381 0.4921
0.5267 0.3467 0.4544


In [31]:
# Colorless
# Layers of the model
model_layers = [8, 8, "M", 16,"M"]
# Create model
model = small3DNet(model_layers, 16)
# Load model
model.load_state_dict(torch.load('model_saves/3D/Colorless/Colorless3D.pth'))
# Put model on gpu
model.cuda()

children = [x for x in model.children()]

colorlesscs = []
colorlessed = []
for x in children[0][0].weight:
    x0 = x[0][0].cpu().detach().numpy().flatten().reshape(1, -1)
    x1 = x[0][1].cpu().detach().numpy().flatten().reshape(1, -1)
    x2 = x[0][2].cpu().detach().numpy().flatten().reshape(1, -1)
    colorlesscs.append((cosine_similarity(x0, x1)[0][0] + cosine_similarity(x0, x2)[0][0] + cosine_similarity(x1, x2)[0][0]) / 3)
    colorlessed.append((euclidean_distances(x0, x1)[0][0] + euclidean_distances(x0, x2)[0][0] + euclidean_distances(x1, x2)[0][0]) / 3)
    
    print(np.round(euclidean_distances(x0, x1)[0][0], 4), np.round(euclidean_distances(x1, x2)[0][0], 4), np.round(euclidean_distances(x0, x2)[0][0], 4))

0.4179 0.4017 0.4815
0.4668 0.4323 0.5909
0.5324 0.3695 0.4458
0.4619 0.4779 0.5326
0.5259 0.5058 0.4969
0.3496 0.541 0.5672
0.5021 0.4527 0.3046
0.4597 0.3234 0.4864


In [30]:
print(np.round(uniformcs, 4))
print(np.round(colorlesscs, 4))
print(np.round(uniformed, 4))
print(np.round(colorlessed, 4))
print(np.sum([colorlessed[i] - uniformed[i] for i in range(len(uniformed))]))
print(np.sum([uniformcs[i] - colorlesscs[i] for i in range(len(uniformcs))]))

[ 0.6552 -0.2351 -0.1178  0.2014  0.0284  0.1572  0.0812  0.3202]
[ 0.2504  0.0421  0.582   0.3846  0.2767 -0.0506  0.2448  0.4763]
[0.3477 0.5515 0.7574 0.3875 0.5715 0.4808 0.5733 0.4426]
[0.4337 0.4967 0.4492 0.4908 0.5095 0.4859 0.4198 0.4232]
-0.40333628654479975
-1.1156923174858093
